In [6]:
cd ..

/Users/satouwataru/Desktop/DiveIntoCode/git/diveintocode-ml/GraduationWork/v3/codes/model


In [7]:
cd ..

/Users/satouwataru/Desktop/DiveIntoCode/git/diveintocode-ml/GraduationWork/v3/codes


In [8]:
cd ..

/Users/satouwataru/Desktop/DiveIntoCode/git/diveintocode-ml/GraduationWork/v3


In [62]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsClassifier
import xgboost as xgb

def predict_toto(year = None):
    
    data = pd.read_csv('data/model/preprocessing.csv', index_col=0).reset_index(drop = True)
    data = data.sort_values('年月日', ascending=False)
    # y_columns = ['y_H_goal', 'y_A_goal', 'y_goal_deff', 'y_even_flg', 'y_H_result']
    y_columns = []
    for col in data.columns:
        if col[:2] == 'y_':
            y_columns += [col]
    
    # train、test、totoに分ける
    train = data[data['train_test'] == 'train'].drop(columns = ['train_test'])
    toto = data[data['train_test'] == 'toto'].drop(columns = ['train_test'])
    train = train.sort_values('年月日', ascending=False).reset_index(drop = True)
    toto = toto.sort_values('年月日', ascending=False).reset_index(drop = True)
    
    # trainの引き分けとその他のデータ数を合わせる
    train_0 = train[train['y_H_result'] == 0]
    train_1 = train[train['y_H_result'] == 1]
    train_2 = train[train['y_H_result'] == 2]

    n_row_1_2 = min([train_1.shape[0], train_2.shape[0]])
    n_row = min([train_0.shape[0], n_row_1_2*2])

    train_0 = train_0.iloc[:n_row]
    train_1 = train_1.iloc[:int(n_row/2)]
    train_2 = train_2.iloc[:int(n_row/2)]
    train = pd.concat([train_0, train_1, train_2])
    
    df_result = toto[['年月日', 'H_Team', 'A_Team']]
    
    '''
    得点の回帰(線形回帰)
    '''
    # y_columns = ['y_H_goal', 'y_A_goal', 'y_goal_deff', 'y_even_flg', 'y_H_result']
    # x_とy_に分ける
    x_train = train.drop(columns = y_columns)
    x_toto = toto.drop(columns = y_columns)
    y_train = train['y_H_goal']
    # 学習
    lr = LinearRegression()
    lr.fit(x_train, y_train)
    #評価の実行
    y_toto = lr.predict(x_toto)
    # 結果を保存
    df_pred = pd.DataFrame(y_toto, columns = ['H_goal_lr'])
    df_result = pd.concat([df_result, df_pred], axis = 1)
    
    '''
    得失点の回帰(線形回帰)
    '''
    # y_columns = ['y_H_goal', 'y_A_goal', 'y_goal_deff', 'y_even_flg', 'y_H_result']
    # x_とy_に分ける
    x_train = train.drop(columns = y_columns)
    x_toto = toto.drop(columns = y_columns)
    y_train = train['y_goal_deff']
    # 学習
    lr = LinearRegression()
    lr.fit(x_train, y_train)
    #評価の実行
    y_toto = lr.predict(x_toto)
    # 結果を保存
    df_pred = pd.DataFrame(y_toto, columns = ['goal_deff_lr'])
    df_result = pd.concat([df_result, df_pred], axis = 1)
    
    '''
    失点の回帰(線形回帰)
    '''
    # y_columns = ['y_H_goal', 'y_A_goal', 'y_goal_deff', 'y_even_flg', 'y_H_result']
    # x_とy_に分ける
    x_train = train.drop(columns = y_columns)
    x_toto = toto.drop(columns = y_columns)
    y_train = train['y_A_goal']
    # 学習
    lr = LinearRegression()
    lr.fit(x_train, y_train)
    #評価の実行
    y_toto = lr.predict(x_toto)
    # 結果を保存
    df_pred = pd.DataFrame(y_toto, columns = ['A_goal_lr'])
    df_result = pd.concat([df_result, df_pred], axis = 1)

    '''
    引き分けその他の二値分類(KNeighborsClassifier)
    '''
    # y_columns = ['y_H_goal', 'y_A_goal', 'y_goal_deff', 'y_even_flg', 'y_H_result']
    # x_とy_に分ける
    x_train = train.drop(columns = y_columns)
    x_toto = toto.drop(columns = y_columns)
    y_train = train['y_even_flg']
    # 学習
    model = KNeighborsClassifier(n_neighbors=2)
    model.fit(x_train, y_train)
    #評価の実行
    y_toto = model.predict(x_toto)
    # 結果を保存
    df_pred = pd.DataFrame(y_toto, columns = ['even_flg_KN'])
    df_result = pd.concat([df_result, df_pred], axis = 1)
    
    '''
    試合結果の三値分類(KNeighborsClassifier)
    '''
    # y_columns = ['y_H_goal', 'y_A_goal', 'y_goal_deff', 'y_even_flg', 'y_H_result']
    # x_とy_に分ける
    x_train = train.drop(columns = y_columns)
    x_toto = toto.drop(columns = y_columns)
    y_train = train['y_H_result']
    # 学習
    model = KNeighborsClassifier(n_neighbors=3)
    model.fit(x_train, y_train)
    #評価の実行
    y_toto = model.predict(x_toto)
    # 結果を保存
    df_pred = pd.DataFrame(y_toto, columns = ['H_result_KN'])
    df_result = pd.concat([df_result, df_pred], axis = 1)
    
    '''
    試合結果の三値分類(XGboost)
    '''
    # y_columns = ['y_H_goal', 'y_A_goal', 'y_goal_deff', 'y_even_flg', 'y_H_result']
    # x_とy_に分ける
    x_train = train.drop(columns = y_columns)
    x_toto = toto.drop(columns = y_columns)
    y_train = train['y_H_result']
    
    dtrain = xgb.DMatrix(x_train, label=y_train)
    dtest = xgb.DMatrix(x_toto)
    
    param = {'objective': 'multi:softmax', 'num_class': 3}
    
    # 学習
    bst = xgb.train(param, dtrain, 1000)
    
    #評価の実行
    y_toto = bst.predict(dtest)
    # 結果を保存
    df_pred = pd.DataFrame(y_toto, columns = ['H_result_XG'])
    df_result = pd.concat([df_result, df_pred], axis = 1)
    
    '''
    引き分けその他の二値分類(XGboost)
    '''
    # y_columns = ['y_H_goal', 'y_A_goal', 'y_goal_deff', 'y_even_flg', 'y_H_result']
    # x_とy_に分ける
    x_train = train.drop(columns = y_columns)
    x_toto = toto.drop(columns = y_columns)
    y_train = train['y_even_flg']
    
    dtrain = xgb.DMatrix(x_train, label=y_train)
    dtest = xgb.DMatrix(x_toto)
    
    param = {'objective': 'multi:softmax', 'num_class': 2}
    
    # 学習
    bst = xgb.train(param, dtrain, 1000)
    
    #評価の実行
    y_toto = bst.predict(dtest)
    # 結果を保存
    df_pred = pd.DataFrame(y_toto, columns = ['even_flg_XG'])
    df_result = pd.concat([df_result, df_pred], axis = 1)
    
    return df_result

In [63]:
predict_toto()

,年月日,H_Team,A_Team,H_goal_lr,goal_deff_lr,A_goal_lr,even_flg_KN,H_result_KN,H_result_XG,even_flg_XG
0,20210505.0,40.0,20.0,1.375866,-0.202974,1.578841,1.0,0.0,0.0,0.0
1,20210505.0,31.0,29.0,1.565792,0.194589,1.371203,0.0,1.0,1.0,0.0
2,20210505.0,38.0,34.0,1.512148,-0.181280,1.693428,0.0,0.0,0.0,1.0
3,20210505.0,37.0,26.0,1.439948,0.014557,1.425391,1.0,0.0,2.0,0.0
4,20210505.0,36.0,28.0,1.389453,0.035950,1.353503,0.0,1.0,2.0,0.0
5,20210505.0,10.0,1.0,2.472871,0.074570,2.398302,0.0,0.0,1.0,0.0
6,20210505.0,8.0,9.0,1.902067,0.377819,1.524248,0.0,0.0,1.0,1.0
7,20210505.0,0.0,18.0,2.758554,0.252315,2.506239,0.0,1.0,1.0,0.0
8,20210505.0,4.0,3.0,1.955331,0.458517,1.496813,0.0,0.0,1.0,0.0
9,20210505.0,14.0,5.0,1.572483,-0.131202,1.703685,0.0,1.0,1.0,0.0


In [6]:
pd.concat([df_result,df_acuracy], axis = 1)

,年月日,H_Team,A_Team,is_even_lgbm,is_even_0_1_lgbm,is_even_1_1_lgbm,is_even_lgbm_acc,is_lose_lgbm,is_lose_0_1_lgbm,is_lose_1_1_lgbm,...,w_e_l_1_2_lgbm,w_e_l_2_2_lgbm,w_e_l_lgbm_acc,goal_deff_lgbm,goal_deff_lgbm_rmse,H_goal_lgbm,H_goal_lgbm_rmse,A_goal_lgbm,A_goal_lgbm_rmse,0
0,20201101.0,40.0,22.0,0.0,0.517855,0.482145,0.442708,0.0,0.722023,0.277977,...,0.429665,0.344232,0.473958,0.107102,1.510898,1.534894,1.137032,0.864063,1.127339,NaN
1,20201101.0,33.0,39.0,1.0,0.472177,0.527823,0.442708,0.0,0.503258,0.496742,...,0.377315,0.396526,0.473958,-0.067474,1.510898,1.311225,1.137032,0.829646,1.127339,NaN
2,20201101.0,25.0,41.0,1.0,0.472411,0.527589,0.442708,0.0,0.700948,0.299052,...,0.486657,0.296261,0.473958,0.390572,1.510898,1.557606,1.137032,0.751652,1.127339,NaN
3,20201101.0,36.0,26.0,1.0,0.469054,0.530946,0.442708,0.0,0.560327,0.439673,...,0.407925,0.353188,0.473958,0.047902,1.510898,1.225003,1.137032,1.029955,1.127339,NaN
4,20201101.0,32.0,27.0,0.0,0.505576,0.494424,0.442708,0.0,0.591624,0.408376,...,0.452884,0.322239,0.473958,0.234438,1.510898,1.522925,1.137032,1.601814,1.127339,NaN
5,20201101.0,35.0,21.0,1.0,0.457609,0.542391,0.442708,0.0,0.514140,0.485860,...,0.329249,0.435178,0.473958,-0.073798,1.510898,1.351944,1.137032,1.083416,1.127339,NaN
6,20201031.0,1.0,15.0,0.0,0.500339,0.499661,0.442708,0.0,0.523654,0.476346,...,0.398877,0.407158,0.473958,0.010810,1.510898,1.351132,1.137032,1.427625,1.127339,NaN
7,20201031.0,19.0,3.0,1.0,0.483529,0.516471,0.442708,1.0,0.405208,0.594792,...,0.334080,0.482137,0.473958,-0.092950,1.510898,1.340565,1.137032,1.833370,1.127339,NaN
8,20201031.0,9.0,8.0,1.0,0.486102,0.513898,0.442708,0.0,0.550414,0.449586,...,0.397965,0.423773,0.473958,0.092725,1.510898,1.424891,1.137032,1.616674,1.127339,NaN
9,20201031.0,2.0,11.0,1.0,0.476583,0.523417,0.442708,0.0,0.540133,0.459867,...,0.381114,0.463194,0.473958,-0.059919,1.510898,1.563272,1.137032,1.425202,1.127339,NaN


In [7]:
df_acuracy

,0
is_even_lgbm,0.442708
is_lose_lgbm,0.479167
w_e_l_lgbm,0.473958
goal_deff_lgbm,1.510898
H_goal_lgbm,1.137032
A_goal_lgbm,1.127339
